To execute this notebook, you need to either
 - Download a pre-trained model
 - Train an example model by excecuting the model_training_tutorial.ipynb notebook

In this tutorial we will learn to:
- Load a previously trained model
- Extract DeepPrint features from fingerprint images
- Evaluate the performance of the extracted fixed-length representations

## Embedding extraction

After training the model, we can extract the DeepPrint features for the fingerprint images. This is done by calling the `extract` method of the `DeepPrintExtractor` class.

In [2]:
import os

from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex, get_DeepPrint_TexMinu, DeepPrintExtractor

# Dimension and number of training subjects must be known to load the pre-trained model

# To load the pre-trained model parameters use num_training_subjects=8000
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=8000, num_dims=256)

# To load the pre-trained model parameters use
MODEL_DIR: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/DeepPrint_TexMinu_512") # Path to the directory containing the model parameters
extractor.load_best_model(MODEL_DIR)

Loaded best model from /home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/DeepPrint_TexMinu_512/best_model.pyt


/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Now we need to specify the dataset, for which we want to extract the embeddings

In [3]:
import os

from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute path to the notebooks/example-dataset directory here! 
DATASET_PATH: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/example-dataset-png")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_PATH),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )

image_dataset: Dataset = Dataset(image_loader, image_loader.ids)

# The second value is for the minutiae branch, which we do not have in this example
texture_embeddings, minutia_embeddings = extractor.extract(image_dataset)

Created IdentifierSet with 10 subjects and a total of 100 samples.


100%|██████████| 4/4 [00:35<00:00,  8.87s/it]


In [4]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)

embeddings._array

Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.


array([[ 0.03856677,  0.05139249, -0.10364334, ...,  0.01890967,
         0.08077914,  0.01240349],
       [ 0.06720702,  0.02505201, -0.08684536, ...,  0.01502696,
         0.08776784, -0.00227975],
       [ 0.04592977,  0.04109466, -0.1052244 , ...,  0.00685132,
         0.08758652,  0.01492492],
       ...,
       [ 0.05089308, -0.03129176, -0.05204975, ...,  0.102452  ,
         0.08917881,  0.17647512],
       [ 0.03145429, -0.04707128, -0.02905202, ...,  0.11360291,
         0.0901756 ,  0.14846209],
       [ 0.03581819, -0.0318948 , -0.05130583, ...,  0.11635151,
         0.09079733,  0.17041668]], dtype=float32)

Waking algo for core to bianry using gray code

In [31]:
import os

def gray_code(n):
    return n ^ (n >> 1)

def convert_to_gray_code(x, y):
    gray_x = gray_code(int(x))
    gray_y = gray_code(int(y))
    return bin(gray_x)[2:], bin(gray_y)[2:]

def pad_binary(binary, length):
    return binary.zfill(length)

def process_txt_file(txt_file, output_folder):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        x, y = map(float, lines[0].split()[:2])  # Read only the first two values
        
        gray_x, gray_y = convert_to_gray_code(x, y)
        
        concatenated_binary = gray_x + gray_y
        
        max_length = len(concatenated_binary)
        
        padded_binary = pad_binary(concatenated_binary, max_length)
        
        output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(txt_file))[0] + '.txt')
        with open(output_file, 'w') as out_f:
            out_f.write(padded_binary)


def process_folder(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            process_txt_file(os.path.join(input_folder, filename), output_folder)

# Specify input and output folders
input_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/example-dataset-pose-points'
output_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/example-dataset-pose-binary'

# Process all .txt files in the input folder
process_folder(input_folder, output_folder)


Hardcored Core points

In [ ]:
import numpy as np

# Define z-score normalization function
def z_score_normalize(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data

# Define function to convert normalized array to binary array
def to_binary_array(data, threshold):
    return (data >= threshold).astype(int)

# Choose normalization method and threshold
normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
threshold = 0.5

# List to store binary arrays
binary_arrays = []

# Normalize and convert each array to binary
for array in texture_embeddings._array:
    # Sort the array (not sure if this step is necessary based on your code)
    sorted(array)

    # Normalize the array
    normalized_array = normalize_method(array)
    
    # Convert the normalized array to binary using the threshold
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append the binary array to the list
    binary_arrays.append(binary_array)

# Generate the permutation pattern from core points (assuming core_points is available)
core_points = [0, 1, 0, 1]  # Example core points as a binary array

# Function to permute binary array in intervals using a pattern
def permute_binary_array_in_intervals(binary_array, pattern):
    permuted_array = []
    interval_length = len(binary_array) // len(pattern)
    for i in range(0, len(binary_array), interval_length):
        segment = binary_array[i:i+interval_length]
        permuted_segment = [segment[j] for j in pattern]
        permuted_array.extend(permuted_segment)
    return np.array(permuted_array)

# Permute all binary arrays using the generated pattern
permuted_arrays = []
for binary_array in binary_arrays:
    permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points)
    permuted_arrays.append(permuted_binary_array)


# Print the permuted binary arrays
for i, permuted_binary_array in enumerate(permuted_arrays):
    print(f"Permuted binary array for array {i}:")
    print(permuted_binary_array)


In [51]:
import numpy as np
import os

# Define z-score normalization function
def z_score_normalize(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data

# Define function to convert normalized array to binary array
def to_binary_array(data, threshold):
    return (data >= threshold).astype(int)

# Choose normalization method and threshold
normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
threshold = 0.3

# List to store binary arrays
binary_arrays = []

# Normalize and convert each array to binary
for array in embeddings._array: # texture features
    # Sort the array (not sure if this step is necessary based on your code)
    sorted(array)

    # Normalize the array
    normalized_array = normalize_method(array)
    
    # Convert the normalized array to binary using the threshold
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append the binary array to the list
    binary_arrays.append(binary_array)

# Folder containing core point files
# core_points_folder = "/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_binary_core"
core_points_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/example-dataset-pose-binary'
# Permute all binary arrays using the core points from files in the folder
permuted_array = []
pose_point_patterns = []
for binary_array in binary_arrays:
    print(binary_array)
    for core_points_file in sorted(os.listdir(core_points_folder)):
    # Read core points from the current file
        with open(os.path.join(core_points_folder, core_points_file), "r") as f:
            core_points_str = f.read().strip()
            print(core_points_str)
            pose_point_patterns.append(core_points_str)
    # # Convert the core points string to a list of integers
    #     core_points = [int(bit) for bit in core_points_str]
    #     print(core_points)
    # # Perform Reed-Solomon encoding on core points
    # rs = reedsolo.RSCodec(10)  # Define parameters as per your requirements
    # encoded_core_points = rs.encode(core_points)

    # # Concatenate the encoded core points to the binary array
    # concatenated_array = np.concatenate((binary_array, encoded_core_points), axis=0)

#     # Function to permute binary array in intervals using the pattern
#     def permute_binary_array_in_intervals(binary_array, pattern):
#         permuted_array = []
#         interval_length = len(binary_array) // len(pattern)
#         for i in range(0, len(binary_array), interval_length):
#             segment = binary_array[i:i+interval_length]
#             permuted_segment = [segment[j] for j in pattern]
#             permuted_array.extend(permuted_segment)
#         return np.array(permuted_array)

#     # Permute the binary array using the read core points
#     permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points_str)
#     permuted_array.append(permuted_binary_array)

# # Print the permuted binary arrays
# for i, permuted_binary_array in enumerate(permuted_array):
#     print(f"Permuted binary array for array {i}:")
#     print(permuted_binary_array)


[1 1 0 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 1 1 0
 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 1 0 1
 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1
 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0
 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0
 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 1
 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1
 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0
 1 1 0 0 1 1 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0
 0 0 1 1 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1
 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1
 1 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1
 0 1 1 1 0 0 1 0 1 1 1 0 

In [55]:
# def transform(binary, pattern):
#     # Convert binary and pattern strings into lists for easier manipulation
#     binary_list = list(binary)
#     pattern_list = list(pattern)
    
#     # Get the lengths of binary and pattern lists
#     n = len(binary_list)
#     m = len(pattern_list)

#     # Initialize variables
#     result = []         # To store the final result
#     skipped_bits = []   # To store bits skipped based on the pattern
#     j = 0               # Pointer to track the current position in the pattern

#     # Iterate through each bit in the binary string
#     for i in range(n):
#         if pattern_list[j] == '1':  
#             # If the current pattern bit is '1', add the corresponding binary bit to the result
#             result.append(binary_list[i])
#         else:
#             # If the current pattern bit is '0', skip the binary bit and store it in skipped_bits
#             skipped_bits.append(binary_list[i])
        
#         # Move to the next pattern bit
#         j += 1  
        
#         # Reset the pattern pointer when the end of the pattern is reached
#         if j == m:
#             j = 0
            
#     # Append all skipped bits to the result at the end
#     result.extend(skipped_bits)
    
#     # Convert the result list back into a string and return it
#     return ''.join(result)

# # Function to handle lists of binary and pattern strings
# def transform_all(binaries, patterns):
#     # Apply the transform function to each pair of binary and pattern
#     results = [
#         transform(binary, pattern) 
#         for binary, pattern in zip(binaries, patterns)
#     ]
#     return results

# # Example input
# binary = binary_arrays
# pattern = pose_point_patterns
# # print(pose_point_patterns)
# # Apply the transformation to each binary-pattern pair
# result = transform_all(binary, pattern)


# # Print the results
# print(result)

TypeError: sequence item 0: expected str instance, numpy.int64 found

In [60]:
def transform(binary, pattern):
    # Convert binary and pattern strings into lists for easier manipulation
    binary_list = list(binary)
    pattern_list = list(pattern)
    
    # Get the lengths of binary and pattern lists
    n = len(binary_list)
    m = len(pattern_list)

    # Initialize variables
    result = []         # To store the final result
    skipped_bits = []   # To store bits skipped based on the pattern
    j = 0               # Pointer to track the current position in the pattern

    # Iterate through each bit in the binary string
    for i in range(n):
        if pattern_list[j] == '1':  
            # If the current pattern bit is '1', add the corresponding binary bit to the result
            result.append(str(binary_list[i]))  # Ensure this is a string
        else:
            # If the current pattern bit is '0', skip the binary bit and store it in skipped_bits
            skipped_bits.append(str(binary_list[i]))  # Ensure this is a string
        
        # Move to the next pattern bit
        j += 1  
        
        # Reset the pattern pointer when the end of the pattern is reached
        if j == m:
            j = 0
            
    # Append all skipped bits to the result at the end
    result.extend(str(bit) for bit in skipped_bits)  # Convert skipped bits to strings
    
    # Convert the result list back into a string and return it
    return ''.join(result)

# Function to handle lists of binary and pattern strings
def transform_all(binaries, patterns):
    # Apply the transform function to each pair of binary and pattern
    results = [
        transform(binary, pattern) 
        for binary, pattern in zip(binaries, patterns)
    ]
    return results

# Example input
binary = binary_arrays
pattern = pose_point_patterns

# Apply the transformation to each binary-pattern pair
result = transform_all(binary, pattern)

# Print the results
# print(len(result))
for x in result:
    print("OUTPUT",x)


OUTPUT 11011101101010001101000110110011100101000101011111000000010010100000111000001000000110110001011110000100010101110010110111000101100000111011111001010100001000001001000101010011100010001000010110000101000110001100100100100000001001010000001000001010000100011000000010100100001010011011101010101100011001010100010010010110000100000011010100011001001111010110101101011000010111000100000100001111100000000100001010100000001001000000000001000111000000101100000110011100000010001010010000000001011110010101101110100011
OUTPUT 11011010110001000010100110011101011111100001010100011101111000010100001110001001011100101011001010000110101000010100001000100100100010010100111000010000010101011100011010001100101100110101110000100110010100000101000001110000010001110000000111100100001010001111110101101001000011011000100001001001001100000010000011010100001100100101110101101101101011101001011100010000001000101111100000100010000100100100000101001000000000000110001110000000101100000110001110000010100010100

In [65]:
# def hamming_distance_numpy(x, y):
#     # Ensure both strings are of the same length
#     assert len(x) == len(y), "Binary strings must be of the same length"
#     return np.sum(np.array(x) != np.array(y))

def hamming_dist(s1, s2):
    if len(s1) != len(s2):
        return -1
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

num_rows = 10
num_cols = 10
hamming_distance_matrix = np.zeros((num_rows, num_cols), dtype=int)

for i in range(num_rows):
    x = result[i]
    for j in range(num_cols):
        y = result[j]
        hamming_distance_matrix[i, j] = hamming_dist(x, y)

In [66]:
for x in hamming_distance_matrix:
    print(x)

[  0 243 232 222 241 247 151 238 240 250]
[243   0  37 253 226 220 230 233 247 221]
[232  37   0 236 235 223 227 242 252 230]
[222 253 236   0 249 267 241 250 236 250]
[241 226 235 249   0 192 258  27 233 163]
[247 220 223 267 192   0 248 203 249  87]
[151 230 227 241 258 248   0 255 229 245]
[238 233 242 250  27 203 255   0 230 174]
[240 247 252 236 233 249 229 230   0 246]
[250 221 230 250 163  87 245 174 246   0]


This is the permutation with core as binary value seed

In [ ]:
import numpy as np
import os

# Define z-score normalization function
def z_score_normalize(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data

# Define function to convert normalized array to binary array
def to_binary_array(data, threshold):
    return (data >= threshold).astype(int)

# Choose normalization method and threshold
normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
threshold = 0.3

# List to store binary arrays
binary_arrays = []

# Normalize and convert each array to binary
for array in texture_embeddings._array:
    # Sort the array (not sure if this step is necessary based on your code)
    sorted(array)

    # Normalize the array
    normalized_array = normalize_method(array)
    
    # Convert the normalized array to binary using the threshold
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append the binary array to the list
    binary_arrays.append(binary_array)

# Folder containing core point files
core_points_folder = "/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/example-dataset-pose-points"

# Permute all binary arrays using the core points from files in the folder
permuted_array = []

for binary_array in binary_arrays:
    for core_points_file in sorted(os.listdir(core_points_folder)):
    # Read core points from the current file
        with open(os.path.join(core_points_folder, core_points_file), "r") as f:
            core_points_str = f.read().strip()


    # Convert the core points string to a list of integers
        core_points = [int(bit) for bit in core_points_str]

    # Function to permute binary array in intervals using the pattern
    def permute_binary_array_in_intervals(binary_array, pattern):
        permuted_array = []
        interval_length = len(binary_array) // len(pattern)
        for i in range(0, len(binary_array), interval_length):
            segment = binary_array[i:i+interval_length]
            permuted_segment = [segment[j] for j in pattern]
            permuted_array.extend(permuted_segment)
        return np.array(permuted_array)

    # Permute the binary array using the read core points
    permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points)
    permuted_array.append(permuted_binary_array)

# Print the permuted binary arrays
for i, permuted_binary_array in enumerate(permuted_array):
    print(f"Permuted binary array for array {i}:")
    print(permuted_binary_array)


Hamming distance code


In [61]:
import numpy as np

# Function to calculate Hamming distance between two binary arrays
def hamming_distance(array1, array2):
    return np.sum(array1 != array2)

# Initialize lists to store binary arrays for each person
binary_arrays_same_person = []
binary_arrays_diff_person = []

# Normalize and convert each array to binary
for array in result:
    # normalized_array = normalize_method(array)
    # binary_array = to_binary_array(normalized_array, threshold)
    binary_array = array
    print(binary_array)
    # Append to appropriate list based on person index
    person_index = i % 10  # Assuming each person has 10 impressions
    if person_index < 10:
        binary_arrays_same_person.append(binary_array)
    else:
        binary_arrays_diff_person.append(binary_array)

# Calculate Hamming distance for same person pairs
print("Hamming distances for same person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(i + 1, len(binary_arrays_same_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")

# Calculate Hamming distance for different person pairs
print("\nHamming distances for different person pairs:")
for i in range(len(binary_arrays_diff_person)):
    for j in range(len(binary_arrays_diff_person)):
        distance = hamming_distance(binary_arrays_diff_person[i], binary_arrays_diff_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")


...
Hamming distances for same person pairs:
Impression 0 vs Impression 1: 1
Impression 0 vs Impression 2: 1
Impression 0 vs Impression 3: 1
Impression 0 vs Impression 4: 1
Impression 0 vs Impression 5: 1
Impression 0 vs Impression 6: 1
Impression 0 vs Impression 7: 1
Impression 0 vs Impression 8: 1
Impression 0 vs Impression 9: 1
Impression 0 vs Impression 10: 1
Impression 0 vs Impression 11: 1
Impression 0 vs Impression 12: 1
Impression 0 vs Impression 13: 1
Impression 0 vs Impression 14: 1
Impression 0 vs Impression 15: 1
Impression 0 vs Impression 16: 1
Impression 0 vs Impression 17: 1
Impression 0 vs Impression 18: 1
Impression 0 vs Impression 19: 1
Impression 0 vs Impression 20: 1
Impression 0 vs Impression 21: 1
Impression 0 vs Impression 22: 1
Impression 0 vs Impression 23: 1
Impression 0 vs Impression 24: 1
Impression 0 vs Impression 25: 1
Impression 0 vs Impression 26: 1
Impression 0 vs Impression 27: 1
Impression 0 vs Impression 28: 1
Impression 0 vs Impression 29: 1
Impress

## Benchmarking

To evaluate the embeddings, we want to run a benchmark on them. For this, we must first specify the type of benchmark, and which comparisons should be run.

In [14]:
from flx.scripts.generate_benchmarks import create_verification_benchmark

NUM_IMPRESSIONS_PER_SUBJECT = 10
benchmark = create_verification_benchmark(
    #subjects=list(range(100, image_dataset.num_subjects + 100)),
    subjects=list(range(image_dataset.num_subjects)),
    impressions_per_subject=list(range(NUM_IMPRESSIONS_PER_SUBJECT))
)

100%|██████████| 10/10 [00:00<00:00, 14711.69it/s]


Now we can run the benchmark. To do this, we must first specify the matcher (in our case cosine similarity of the embeddings)

In [15]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)
matcher = CosineSimilarityMatcher(EmbeddingLoader.combine(texture_embeddings, minutia_embeddings))

results = benchmark.run(matcher)

print(f"Equal-Error-Rate: {results.get_equal_error_rate()}")

Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.


100%|██████████| 1450/1450 [00:00<00:00, 98243.13it/s]

Equal-Error-Rate: 0.0


To visualize the results, we can plot a DET curve. (Do not wonder if it is empty, probably the model is not trained enough. Take a look at the EER instead.)

In [16]:
from flx.visualization.plot_DET_curve import plot_verification_results

figure_path = "DET_curve"

# Lists are used to allow for multiple models to be plotted in the same figure
plot_verification_results(figure_path, results=[results], model_labels=["DeepPrint_TexMinu"], plot_title="example-dataset - verification")



ImportError: cannot import name 'common_texification' from 'matplotlib.backends.backend_pgf' (/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/backends/backend_pgf.py)